# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели *кредитного скоринга* — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Описание данных:**   
 - количество детей в семье      
 - общий трудовой стаж в днях   
 - возраст клиента в годах   
 - уровень образования клиента   
 - идентификатор уровня образования   
 - семейное положение   
 - идентификатор семейного положения   
 - пол клиента   
 - тип занятости   
 - имел ли задолженность по возврату кредитов   
 - ежемесячный доход   
 - цель получения кредита

## Предобработка данных

In [ ]:
import pandas as pd
from pymystem3 import Mystem
from collections import Counter
m = Mystem()

In [1]:
data = pd.read_csv('_')

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Из 12 столбиков с данными пропущенные значения есть только в столбиках days_employed и total_income. Вероятнее всего это связано с тем, что не все кто брал кредиты работали.

### Обработка пропусков

Заменяем пустые значения ежемесячного дохода на медианное, т.к. разрыв между max и min ежемесячного дохода может быть неограниченно большим

In [2]:
data['total_income'] = data['total_income'].fillna(value=(data['total_income'].median())) 

In [3]:
temp = data.copy()
temp['days_employed > 0'] = data['days_employed'] > 0
temp.pivot_table(index='income_type',columns='days_employed > 0', values='debt', aggfunc='count')
display(temp)

ata['days_employed'] = data['days_employed'].abs()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed > 0
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,False
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,False
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,False
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,False
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем,False
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем,True
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость,False
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля,False


In [4]:
data['days_employed'] = data['days_employed']/24

data['days_employed'] = data['days_employed'].fillna(value=(data['days_employed'].mean()))

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Во всех столбцах таблицы теперь равное количество значений, соответсвенно пропусков не осталось, цель достигнута.

### Замена типа данных

In [5]:
data['days_employed'] = data['days_employed'].astype(int)

data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,351,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,167,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,234,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,171,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,38,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,119,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,6,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,288,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,91,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Количество дней сдели целочисленным, т.к. отработать можно только рабочий день полностью.

### Обработка дубликатов

In [6]:
data['education'] = data['education'].str.lower()
data['family_status'] = data['family_status'].str.lower()

data = data.drop_duplicates().reset_index(drop=True)

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
children            21454 non-null int64
days_employed       21454 non-null int64
dob_years           21454 non-null int64
education           21454 non-null object
education_id        21454 non-null int64
family_status       21454 non-null object
family_status_id    21454 non-null int64
gender              21454 non-null object
income_type         21454 non-null object
debt                21454 non-null int64
total_income        21454 non-null float64
purpose             21454 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


После удаления явных дубликатов осталось 21 454 значение.

### Лемматизация

In [7]:
def lemmatize(purpose):
    lemma = m.lemmatize(purpose)
    if 'автомобиль' in lemma:
        return 'автомобиль'
    elif 'свадьба' in lemma:
        return 'свадьба'
    elif ('недвижимость') or ('жилье') in lemma:
        return 'нежвижимость'
    elif 'образование' in lemma:
        return 'образование'
    else:
        return 'другое'
        
data['purpose_category'] = data['purpose'].apply(lemmatize)

data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category
0,1,351,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,нежвижимость
1,1,167,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,автомобиль
2,0,234,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,нежвижимость
3,3,171,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,нежвижимость
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,свадьба
5,0,38,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья,нежвижимость
6,0,119,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем,нежвижимость
7,0,6,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование,нежвижимость
8,2,288,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы,свадьба
9,0,91,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи,нежвижимость


Результатом леммитизации стало понимание, что целей кредитования в реальной жизни не так много как могло показаться и можно выделить 3 крупные категории.

### Категоризация данных

Результатом леммитизации стало выделение трёх категорий по целям кредита. Можно оценить объём каждой из категориию

In [8]:
display(data['purpose_category'].value_counts())

print(data['total_income'].describe())

def income_category(income):
    if income <= 65000:
        return 'доход ниже среднего'
    elif 65000 < income <= 165000:
        return 'средний класс'
    else:
        return 'доход выше среднего'

data['income_category'] = data['total_income'].apply(income_category)
display(data['income_category'].value_counts()) 

glossary_education_id = data[['education', 'education_id']]
glossary_education_id = glossary_education_id.drop_duplicates().reset_index(drop=True)
display(glossary_education_id)

glossary_family_status_id = data[['family_status', 'family_status_id']]
glossary_family_status_id = glossary_family_status_id.drop_duplicates().reset_index(drop=True)
display(glossary_family_status_id)

нежвижимость    14824
автомобиль       4306
свадьба          2324
Name: purpose_category, dtype: int64

Минимальный ежемесячный доход 20667.26379327158
Медиана ежемесячного дохода 165226.14396668624
Максимальный ежемесячный доход 2265604.028722744


средний класс          12649
доход выше среднего     7707
доход ниже среднего     1098
Name: income_category, dtype: int64

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,не женат / не замужем,4


Исходя из категорий по целям кредита чаще всего клиенты банка берут кредиты на недвижимость (как личную, так и коммерческую).
У большей части клиентов кредитного отдела банка ежемесячный доход от 65 до 165 тысяч рублей.
Созданы 2 словаря для распознования id семейных статусов и уровней образования.

## Ответы на вопросы заказчика 

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [9]:
data.groupby('children')['debt'].mean()

data.loc[(data['children']==(-1)), 'children'] = 1
data.loc[(data['children']==(20)), 'children'] = 2

def children_category(children):
    if children == 0:
        return 'нет детей'
    elif 1 <= children <= 2:
        return '1-2 ребёнка'
    else:
        return 'многодетная семья'

data['children_category'] = data['children'].apply(children_category)
data.groupby('children_category')['debt'].mean()

children_category
1-2 ребёнка          0.092654
многодетная семья    0.081579
нет детей            0.075438
Name: debt, dtype: float64

**Вывод**

При рассмотрении категоризации по количеству детей заёмщиков наибольшая доля должников с задержками по выплатам в категории "1-2 ребёнка"; наименьшая доля должников с задержками по выплатам в категории тех, у кого нет детей.</div>

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [10]:
data.groupby('family_status_id')['debt'].mean()

data.pivot_table(index=['family_status_id', 'family_status'], values='debt')

,,debt
family_status_id,family_status,
0,женат / замужем,0.075452
1,гражданский брак,0.093471
2,вдовец / вдова,0.065693
3,в разводе,0.071130
4,не женат / не замужем,0.097509


**Вывод**

 При рассмотрении информации о влиянии семейного статуса на наличие задержек по выплатам наибольшая доля должников с задержками по выплатам у тех, кто не женат/не замужем; наименьшая доля должников с задержками по выплатам у вдов/вдовцов.</div>

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [11]:
display(data.groupby('income_category')['debt'].mean())

income_category
доход выше среднего    0.077851
доход ниже среднего    0.066485
средний класс          0.084434
Name: debt, dtype: float64

**Вывод**

 При рассмотрении информации о влиянии дохода на наличие задержек по выплатам наибольшая доля должников с задержками по выплатам у тех, кто относится к среднему классу (от 65 до 165 тысяч рублей); наименьшая доля должников с задержками в категории "доход ниже среднего" (меньше 65 тысяч рублей).</div>

- Как разные цели кредита влияют на его возврат в срок?

In [12]:
display(data.groupby('purpose_category')['debt'].mean())

purpose_category
автомобиль      0.093590
нежвижимость    0.077712
свадьба         0.080034
Name: debt, dtype: float64

**Вывод**

При рассмотрении информации о влиянии целей кредита на наличие задержек по выплатам наибольшая доля должников с задержками по выплатам у тех, чья цель кредита - автомобиль; наименьшая доля должников с задержками в категории "недвижимость".</div>

## Общий вывод

Портрет хорошего заёмщика: вдовец/вдова без детей с ежемесячным доходом меньше 65 тысяч рублей,берущий кредит на недвижимость (прям грустно).   
Портрет плохого заёмщика: жентатый/замужний человек с 1-2 детьми с ежемесячным доходом между 65 и 165 тысячами рублей,берущий кредит на автомобиль (это нужно сказать голосом Якубовича).